In [1]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
from sklearn.pipeline import make_pipeline

In [5]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

2023/06/14 21:17:28 INFO mlflow.tracking.fluent: Experiment with name 'green-taxi-duration' does not exist. Creating a new experiment.


<Experiment: artifact_location='gs://mlops-data-2023/1', creation_time=1686777448749, experiment_id='1', last_update_time=1686777448749, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [6]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [7]:
df_train = read_dataframe('/home/harsha/notebooks/data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('/home/harsha/notebooks/data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [10]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


In [11]:
from mlflow.tracking import MlflowClient


In [12]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = 'c9e45bb0ab4e4856b85c6b91c7c51b61'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [13]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

/tmp/ipykernel_3640/2836594815.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')


MlflowException: The following failures occurred while downloading one or more artifacts from gs://mlops-data-2023/1/c9e45bb0ab4e4856b85c6b91c7c51b61/artifacts: {'dict_vectorizer.bin': 'NotFound("GET https://storage.googleapis.com/download/storage/v1/b/mlops-data-2023/o/1%2Fc9e45bb0ab4e4856b85c6b91c7c51b61%2Fartifacts%2Fdict_vectorizer.bin?alt=media: No such object: mlops-data-2023/1/c9e45bb0ab4e4856b85c6b91c7c51b61/artifacts/dict_vectorizer.bin: (\'Request failed with status code\', 404, \'Expected one of\', <HTTPStatus.OK: 200>, <HTTPStatus.PARTIAL_CONTENT: 206>)")'}

In [ ]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [ ]:
dv